In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim
import numpy as np
import random
import tqdm

## Create Dataset

In [2]:
text ="""Weil Bananen böse sind und sie krumme Sachen machen
Weil sie Leute um die Ecke bringen
Weil ihr gerade Sachen viel zu Mainstream waren
Weil der Erderwärmung
Weil die Iluminaten das so wollen
Weil sie Langeweile hatte
Weil wegen halt ist so
Weil sie wenn sie gerade wär ja keine Banane wär
Weil sie wegen übermäßiger Masturbation keine Erektion mehr verspürt
Weil sie Jahre lang einen Bogen um die DDR machte
Weil einer auf dem Acker wetzte und sich auf die Banane setzte
Weil die Affen dran gefummelt haben
Es gab Bemühungen in der DDR Bananen mittels kubanischer Spreewaldgurken nachzubauen aus Verzweiflung haben sich die Bananen deshalb krumm gemacht
Weil die Affen im Urwald drauf schaukeln tun
Weil sie seit Jahrtausenden im Urwald Limbo tanzen dadurch ist der Rücken der Bananen ziemlich krumm geworden
Weil sie sonst nicht in die Schale passt
Weil sie durch die Bananenbatterie so geformt wurde
Das kommt vom Mond
Auch Bananen haben das Bedürfnis ihren unteren Teil zu begutachten  Ein nach vorne beugen ist dabei zwingend
Eine andere Theorie besagt dass sie sich aufgrund der Erdanziehung im Laufe der Zeit krümmen
Bananen sind seit dem Bananenkrieg von krumm
Bananen sind krumm weil die Raum Zeit sich biegt und die Bananen biegen sich mit
Weil sie von der Sonne gekitzelt wurde
Weil sie zu dumm war sich gerade zu biegen
Weil sie beim Sit Up machen erstarrt sind
Weil sie zu viel Unsinn angestellt haben
Weil sie verstecken gespielt hat mit anderen Bananen und und hat dabei versucht sich kleine zu machen was leider nicht ganz klappte
Weil niemand in den Urwald zog und die Banane gerade bog
Weil Baum
Weil sie sich krumm gelacht haben
Weil man sie dann so viel besser als Spielzeugpistole verwenden kann
Damit sie leichter an das männliche Glied erinnert
Weil sie gelb sind
Albert Einstein sagte Große Massen krümmen den Raum Die Erde hat eine große Masse und deshalb krümmt sie den Raum um die Banane herum Die Krummheit der Banane ist dieser Theorie also nur eine Illusion denn innerhalb des gekrümmten Raums ist die Banane natürlich völlig gerade oder so
Weil sie als KanonenFutter benuzt wurde
Weil man sie nicht nur zum Essen verwendet hat
Weil sie es kann"""

In [3]:
sencentce = text.lower().split()
trigrams = [([sencentce[i-1], sencentce[i +1]], sencentce[i]) for i in range(1, len(sencentce) - 1)]
print(trigrams[:3])
vocab = list(set(sencentce))
vocab.append('-')
word_to_ix = {word: i for i, word in enumerate(vocab)}

[(['weil', 'böse'], 'bananen'), (['bananen', 'sind'], 'böse'), (['böse', 'und'], 'sind')]


In [4]:
hidden_dim = 256
class NGramLanguage(nn.Module):
    
    def __init__(self, vocab_sice, emb_size, context_size):
        super(NGramLanguage, self).__init__()
        self.embedding = nn.Embedding(vocab_sice, emb_size)
        self.linear1 = nn.Linear(context_size * emb_size, hidden_dim)
        self.linear2 = nn.Linear(hidden_dim, vocab_sice)
        
    def forward(self, x):
        emb = self.embedding(x).view((1, -1))
        out = F.relu(self.linear1(emb))
        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs

In [5]:
learning_rate = 0.01
losses = []
loss_function = nn.NLLLoss()
model = NGramLanguage(len(vocab), 10, 2)
op = optim.SGD(model.parameters(), lr=learning_rate)

In [6]:
for epoch in range(25):
    total_loss = 0
    iterator = tqdm.tqdm(trigrams)
    for context, target in iterator:
        context_ids = torch.tensor([word_to_ix[w] for w in context], dtype=torch.long)
        model.zero_grad()
        log_probs = model(context_ids)
        loss = loss_function(log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long))
        loss.backward()
        op.step()
        total_loss += loss.item()
        iterator.set_postfix(loss=float(loss))
    losses.append(total_loss)

100%|██████████| 368/368 [00:00<00:00, 370.42it/s, loss=0.439]


In [7]:
model.eval()

NGramLanguage(
  (embedding): Embedding(191, 10)
  (linear1): Linear(in_features=20, out_features=256, bias=True)
  (linear2): Linear(in_features=256, out_features=191, bias=True)
)

In [8]:
with torch.no_grad():
    context = ['-', 'männliche']
    context_ids = torch.tensor([word_to_ix[w] for w in context], dtype=torch.long)
    pred = model(context_ids)
    print(pred)
    index_of_prediction = np.argmax(pred)
    print(vocab[index_of_prediction])

tensor([[ -2.9355,  -6.8245,  -8.3649, -10.4258, -10.4433,  -9.9690,  -8.4168,
         -12.4639,  -8.7139,  -7.2109,  -7.3466,  -7.8919,  -7.3603,  -8.5415,
          -5.4777,  -8.3989,  -8.8717,  -8.0988,  -5.5351,  -5.1989,  -8.3598,
          -9.5222, -11.0208,  -4.3097,  -7.3706, -10.1297,  -9.1615, -10.3087,
          -7.8628,  -9.7942,  -7.1953,  -9.3680,  -4.1939, -10.9322,  -6.1546,
          -4.8654, -10.3082,  -6.7962,  -9.8922,  -7.6840, -10.1978,  -7.9522,
          -7.1923,  -7.7577,  -8.2595,  -7.5430,  -7.4196,  -9.0294,  -5.7491,
          -8.1743,  -8.1038,  -9.2269, -10.6906, -10.2761,  -6.9589, -10.1868,
          -6.9804,  -9.1068,  -7.2283,  -4.2672,  -8.7693,  -9.2917,  -4.6886,
          -7.6183,  -1.9595,  -6.8424,  -8.6870,  -8.5467,  -2.3191,  -8.2937,
          -8.1097,  -8.7379,  -9.5606,  -6.7030,  -7.9353,  -9.9872,  -2.6664,
          -6.9691, -11.4396,  -6.9002,  -7.6009,  -8.9282,  -9.4039,  -7.0506,
          -5.6116,  -7.4902,  -5.4555,  -9.4438,  -8

In [9]:
class WordVec():
    def __init__(self, emb):
        super(WordVec, self).__init__()
        self.embedding = emb
        
        
    def forward(self, x):
        emb = self.embedding(x).view((1, -1))
        return emb

In [10]:
with torch.no_grad():
    word_vec = WordVec(model.embedding)
    context= ['-', 'ddr']
    context_ids = torch.tensor([word_to_ix[w] for w in context], dtype=torch.long)
    pred = model(context_ids)
    print(pred)

tensor([[ -4.4048,  -8.2368, -10.6210, -11.1354, -10.5664, -11.6668, -11.2160,
         -10.6912,  -9.5073, -11.2875, -11.5766, -10.5343,  -6.6726,  -7.9250,
         -14.3781, -10.3146, -10.3807,  -9.2874,  -9.4006, -10.3426,  -7.7176,
         -10.5697, -12.1443,  -7.3634, -13.1275, -11.9956, -12.6232, -10.4637,
         -13.2394,  -9.1006,  -2.9371, -12.2432, -10.9642,  -8.2179, -11.2230,
          -9.8347, -11.9225,  -6.2976, -10.6430,  -5.9998, -10.0068,  -8.6215,
         -10.2026, -11.0080,  -8.8340, -11.2243, -11.4376, -12.4624,  -6.8494,
          -9.4855,  -9.6710, -15.5552, -12.4407, -11.5040, -10.1961,  -6.5366,
          -7.9872, -10.8028,  -8.2779,  -9.0159,  -9.6937, -10.4606,  -5.6278,
          -9.6047,  -7.2292,  -9.1991,  -9.9766,  -6.8501,  -8.1059, -10.2993,
          -9.2062, -10.5814, -10.4844, -10.9201,  -7.8983, -11.3646,  -0.4324,
          -8.8614, -13.1491, -10.3902,  -9.5909,  -9.1018,  -9.6424,  -6.8304,
          -6.6116, -10.2972,  -8.2575, -11.4330,  -9